In [2]:
import cv2
import numpy as np
import os
import glob
from scipy.cluster.vq import *
from scipy.spatial import distance
import matplotlib.pyplot as plt
import time

import sys
sys.path.append("/home/hank/libsvm-3.24/python")
from svmutil import *

from utilities import get_data, plot_heatmap, plot_res

## Detect train data feature

In [2]:
des_list = []
path = "hw5_data/train/**/*"

t1 = time.time()
files = glob.glob(path)
sift = cv2.xfeatures2d.SIFT_create()
for File in files:
    im = cv2.imread(File)
    #im = cv2.resize(im, (200,200), interpolation = cv2.INTER_CUBIC)
    kp1, des1 = sift.detectAndCompute(im,None)
    des_list.append((File, des1))
    #print(len(des1))

descriptors = des_list[0][1]
for image_path, descriptor in des_list[1:]:
    if descriptor is None:
        print(0)
        continue
    #print(descriptor.shape)
    descriptors = np.vstack((descriptors, descriptor))
    #print(descriptor.shape)
#print(descriptors)
t2 = time.time()
print("time:", t2-t1)

print(descriptors.shape)

time: 57.40946102142334
(762332, 128)


## Detect test data feature

In [3]:
test_list = []
path = "hw5_data/test/**/*"

t1 = time.time()
files = glob.glob(path)
sift = cv2.xfeatures2d.SIFT_create()
for File in files:
    im = cv2.imread(File)
    #im = cv2.resize(im, (200,200), interpolation = cv2.INTER_CUBIC)
    kp1, des1 = sift.detectAndCompute(im,None)
    test_list.append((File, des1))
    #print(des1)
    
t2 = time.time()
print("time:", t2-t1)

time: 1.4496216773986816


## Enlarge feature number

In [2]:
des_list = []
path = "hw5_data/train/**/*"

t1 = time.time()
files = glob.glob(path)
sift = cv2.xfeatures2d.SIFT_create()
for File in files:
    im = cv2.imread(File)
    im = cv2.resize(im, (600,600), interpolation = cv2.INTER_CUBIC)
    kp1, des1 = sift.detectAndCompute(im,None)
    des_list.append((File, des1))
    #print(len(des1))

descriptors = des_list[0][1]
for image_path, descriptor in des_list[1:]:
    if descriptor is None:
        print(0)
        continue
    #print(descriptor.shape)
    descriptors = np.vstack((descriptors, descriptor))
    #print(descriptor.shape)
#print(descriptors)
t2 = time.time()
print("time:", t2-t1)

time: 333.06759428977966


In [3]:
test_list = []
path = "hw5_data/test/**/*"

t1 = time.time()
files = glob.glob(path)
sift = cv2.xfeatures2d.SIFT_create()
for File in files:
    im = cv2.imread(File)
    im = cv2.resize(im, (1000,1000), interpolation = cv2.INTER_CUBIC)
    kp1, des1 = sift.detectAndCompute(im,None)
    test_list.append((File, des1))
    #print(des1)
    
t2 = time.time()
print("time:", t2-t1)

time: 16.012665033340454


## K means for all feature

In [4]:
# Perform k-means clustering
t1 = time.time()
k = 300
voc, variance = kmeans(descriptors, k, 1) 
t2 = time.time()
print("time:", t2-t1)

time: 1140.9546167850494


In [10]:
voc

array([[18.283516 , 24.980072 , 26.57922  , ..., 47.75518  , 53.509296 ,
        22.055067 ],
       [98.815475 , 23.157852 ,  5.405956 , ...,  6.8360395,  5.0561466,
        10.218122 ],
       [18.010374 , 55.612926 , 82.49174  , ...,  5.682703 ,  9.713827 ,
        39.472733 ],
       ...,
       [50.21564  , 85.94494  , 39.843777 , ..., 14.389107 , 17.855045 ,
        21.147003 ],
       [13.7968645, 10.390819 , 12.190155 , ..., 10.519499 ,  7.637784 ,
         9.218813 ],
       [18.49173  , 16.745148 , 16.463459 , ..., 13.846076 , 13.017975 ,
        18.43502  ]], dtype=float32)

## Histogram of features based on K means center for each training image

In [5]:
# Calculate the histogram of features
im_features = np.zeros((1500, k), "float32")
for i in range(1500):
    if des_list[i][1] is None:
        continue
    words, distance = vq(des_list[i][1],voc)

    for w in words:
        im_features[i][w] += 1
        
    im_features[i] = (im_features[i] - np.mean(im_features[i])) / np.std(im_features[i])
    
print(im_features)

[[-0.16075125  0.35780126 -0.42002752 ... -0.67930377  0.6170775
   0.35780126]
 [-0.47099116 -0.92826414  0.44355482 ...  0.44355482  1.3581008
  -0.01371818]
 [-0.6330898   0.35508364 -1.0283592  ... -0.2378204   4.703047
   0.15744895]
 ...
 [-0.02461809 -1.1107069  -0.45905364 ...  0.19259968  1.0614707
  -0.45905364]
 [-0.38540205  1.6160198   0.38437563 ... -1.0012242   0.07646455
   0.38437563]
 [-0.36858153  1.5415338   0.48035866 ... -0.36858153  0.05588856
  -0.36858153]]


## Histogram of features based on K means center for each testing image

In [6]:
# Calculate the histogram of features
test_features = np.zeros((150, k), "float32")
for i in range(150):
    if test_list[i][1] is None:
        continue
    words, distance = vq(test_list[i][1],voc)
    
    for w in words:
        test_features[i][w] += 1
    test_features[i] = (test_features[i] - np.mean(test_features[i])) / np.std(test_features[i])

print(test_features)

[[ 0.9034093  -1.0657126  -0.26016268 ... -0.70769036  1.7089591
  -0.08115161]
 [-0.9183615  -1.2459577  -0.263169   ...  1.047216   -0.263169
   0.06442728]
 [-0.6311171  -0.6311171   0.9401289  ... -0.10736844  0.41638023
   1.4638776 ]
 ...
 [-0.05533787  0.4145118   0.25789526 ...  0.10127869  1.3542112
  -0.52518755]
 [ 0.5408422  -1.0656196  -0.26238868 ... -0.4631964  -0.86481184
   0.5408422 ]
 [-0.04705126 -0.04705126 -0.44466752 ... -0.44466752 -0.24585938
  -0.64347565]]


In [12]:
import pickle
with open('vocab.pkl', 'wb') as handle:
    pickle.dump(voc, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('train_image_feats_1.pkl', 'wb') as handle:
    pickle.dump(im_features, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('tEST_image_feats_1.pkl', 'wb') as handle:
    pickle.dump(test_features, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [3]:
train_x, train_y, test_x, test_y = get_data(gray=False)

train_y = np.array([np.argmax(x) for x in train_y]).reshape(-1)
test_y = np.array([np.argmax(x) for x in test_y]).reshape(-1)

In [4]:
sift = cv2.xfeatures2d.SIFT_create()

In [5]:
bag_of_features = []
for im in train_x:
    kp, des = sift.detectAndCompute(im,None)
    bag_of_features.append(des)
bag_of_features = np.concatenate(bag_of_features, axis=0).astype('float32')

In [ ]:
t1 = time.time()
voc,_ = kmeans2(bag_of_features, 300, iter=1,minit='++')
t2 = time.time()
print("time:", t2-t1)

In [ ]:
im_features = []

for im in train_x:
    kp, des = sift.detectAndCompute(im,None)
    dist = distance.cdist(voc, des, metric='euclidean')
    idx = np.argmin(dist, axis=0)
    hist, bin_edges = np.histogram(idx, bins=len(voc))
    hist_norm = [float(i)/sum(hist) for i in hist]
    im_features.append(hist_norm)
    
im_features = np.asarray(im_features)

In [ ]:
test_features = []

for im in test_x:
    kp, des = sift.detectAndCompute(im,None)
    dist = distance.cdist(voc, des, metric='euclidean')
    idx = np.argmin(dist, axis=0)
    hist, bin_edges = np.histogram(idx, bins=len(voc))
    hist_norm = [float(i)/sum(hist) for i in hist]
    test_features.append(hist_norm)
    
test_features = np.asarray(test_features)

In [ ]:
import pickle
with open('vocab2.pkl', 'wb') as handle:
    pickle.dump(voc, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('train_image_feats_2.pkl', 'wb') as handle:
    pickle.dump(im_features, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('tEST_image_feats_2.pkl', 'wb') as handle:
    pickle.dump(test_features, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
print('Kernel: linear')
params = '-q -t 0 -s 0 -c 32'
model = svm_train(train_y,im_features,params)
pred, acc, _ = svm_predict(test_y,test_features, model)
print(acc[0])

In [45]:
c_begin,c_end,c_step = (-5,5,1) # Cost
best_acc = 0
best_params = 0
for c in range(c_begin,c_end+c_step,c_step):
    
    # Linear kernel
    params = '-q -v 5 -s 0 -t 0 -c {}'.format(2**c)
    acc = svm_train(train_y,im_features,params)
    if acc > best_acc:
        best_acc = acc
        best_params = 2**c
    print('[Linear] {} {} (best: c={} acc={})'.format(c,acc,best_params,best_acc))

Cross Validation Accuracy = 57.1333%
[Linear] -5 57.13333333333333 (best: c=0.03125 acc=57.13333333333333)
Cross Validation Accuracy = 54.8%
[Linear] -4 54.800000000000004 (best: c=0.03125 acc=57.13333333333333)
Cross Validation Accuracy = 55.4%
[Linear] -3 55.400000000000006 (best: c=0.03125 acc=57.13333333333333)
Cross Validation Accuracy = 57.0667%
[Linear] -2 57.06666666666666 (best: c=0.03125 acc=57.13333333333333)
Cross Validation Accuracy = 56.2667%
[Linear] -1 56.266666666666666 (best: c=0.03125 acc=57.13333333333333)
Cross Validation Accuracy = 56.8667%
[Linear] 0 56.86666666666667 (best: c=0.03125 acc=57.13333333333333)
Cross Validation Accuracy = 55.9333%
[Linear] 1 55.93333333333334 (best: c=0.03125 acc=57.13333333333333)
Cross Validation Accuracy = 54.2%
[Linear] 2 54.2 (best: c=0.03125 acc=57.13333333333333)
Cross Validation Accuracy = 56.2667%
[Linear] 3 56.266666666666666 (best: c=0.03125 acc=57.13333333333333)
Cross Validation Accuracy = 56.4667%
[Linear] 4 56.46666666

In [ ]:
print('Kernel: linear')
params = '-q -t 0 -s 0 -c 700'
model = svm_train(train_y,im_features,params)
pred2, acc2, _ = svm_predict(test_y,test_features, model)